In [22]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, XMLOutputParser
from langchain.output_parsers import YamlOutputParser

In [2]:
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')

##### Invoking the model for generation:

In [ ]:
model = ChatGroq(model="gemma2-9b-it")
print(model.invoke("My name is Namrata.").content)

Hello Namrata! It's nice to meet you. 😊  

What can I do for you today?



##### Creating a chain with prompt and model. Invoking the chain for generation:

In [19]:
prompt = ChatPromptTemplate.from_messages([
    ("system","You are an expert AI Engineer. Provide me answer based on the question"),
    ("user","{input}")
])

chain = prompt | model
result = chain.invoke(input={'input':'My name is Namrata.'})
print(result.content)

Hello Namrata! 

It's nice to meet you.  

Is there anything specific I can help you with? As an AI engineer, I can assist with a variety of tasks, such as:

* **Answering your questions** to the best of my ability based on my training data.
* **Generating creative content** like stories, poems, or even code.
* **Summarizing text** or providing insights from a given piece of writing.
* **Translating languages**.
* **And much more!** 

Just let me know how I can be of service. 😊  




##### Adding different parsers to the chain. Invoking the chain (with parsers) for generation:

StrOutputParser:

In [20]:
str_op_parser = StrOutputParser()
chain = prompt | model | str_op_parser
result = chain.invoke(input={'input':'What is Langsmith?'})
print(result)

Langsmith is an open-source platform designed to simplify the process of fine-tuning large language models (LLMs) for specific tasks. 

Think of it as a toolbox specifically built for tailoring powerful AI models like those from OpenAI (e.g., GPT-3) to your unique needs.

Here's a breakdown of what makes Langsmith special:

**Key Features:**

* **User-Friendly Interface:** Langsmith provides a web-based interface that makes fine-tuning accessible even to users without extensive coding experience.
* **Streamlined Workflow:** It automates many of the complex steps involved in fine-tuning, simplifying the process and reducing the time and effort required.
* **Data Management:** Langsmith helps you manage your training data efficiently, allowing you to upload, format, and split it for fine-tuning.
* **Model Selection:** You can choose from a variety of pre-trained LLMs to fine-tune, depending on your specific task and requirements.
* **Evaluation and Monitoring:** Langsmith offers tools to

JsonOutputParser:

In [21]:
#OPTION 1: Provide the output format info in the system prompt itself.

prompt_json = ChatPromptTemplate.from_messages([
    ("system","You are an expert AI Engineer. Provide me answer based on the question. Response should be in JSON object only."),
    ("user","{input}")
])
str_op_parser = StrOutputParser()
chain = prompt_json | model | str_op_parser
result = chain.invoke(input={'input':'What is Langsmith?'})
print(result)

```json
{
  "description": "Langsmith is an open-source platform designed to simplify the development and deployment of language models.",
  "features": [
    "Streamlined development workflow",
    "Model training and evaluation tools",
    "Integration with popular frameworks (e.g., HuggingFace)",
    "Collaborative environment for model sharing and experimentation"
  ],
  "creator": "The Langsmith team",
  "website": "https://www.langsmith.com/"
}
```


In [26]:
#OPTION 2: Using the JsonOutputParser to decide the output format, instead of mentioning it explicitely in the system prompt. Using PromptTemplate

json_op_parser = JsonOutputParser()
output_format = json_op_parser.get_format_instructions()

json_prompt = PromptTemplate(
    template = "You are an expert AI Engineer. Provide me answer based on the question. \n {format_instruction} \n {input}",
    input_variables=["input"],
    partial_variables={'format_instruction':output_format}

)
print('Prompt:\n ', json_prompt)

chain = json_prompt | model | json_op_parser
result = chain.invoke(input={'input':'What is Langsmith?'})
print('\n',result)

Prompt:
  input_variables=['input'] input_types={} partial_variables={'format_instruction': 'Return a JSON object.'} template='You are an expert AI Engineer. Provide me answer based on the question. \n {format_instruction} \n {input}'

 {'name': 'Langsmith', 'description': 'Langsmith is an open-source platform for developing, deploying, and managing large language models (LLMs).', 'key_features': ['Modular design allows for easy customization and integration with existing workflows.', 'Supports various LLM architectures and frameworks.', 'Provides tools for fine-tuning, evaluation, and monitoring of LLMs.', 'Offers a user-friendly interface for interacting with LLMs and building applications.', 'Focuses on reproducibility and transparency in LLM development.'], 'website': 'https://www.langsmith.ai/'}


In [30]:
#OPTION 3: Using the JsonOutputParser to decide the output format, instead of mentioning it explicitely in the system prompt. Using ChatPromptTemplate

json_op_parser = JsonOutputParser()
output_format = json_op_parser.get_format_instructions()

prompt_json = ChatPromptTemplate.from_messages(
    messages=[
            ("system","You are an expert AI Engineer. Provide me answer based on the question. \n {format_instruction}"),
            ("user","{input}")
        ]
    )

print('Prompt:\n ', prompt_json)

chain = prompt_json | model | json_op_parser
result = chain.invoke(input={'input':'What is Langsmith?','format_instruction':{output_format}})
print(result)

Prompt:
  input_variables=['format_instruction', 'input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instruction'], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answer based on the question. \n {format_instruction}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]
{'name': 'LangSmith', 'description': 'LangSmith is an open-source platform for building and deploying AI models, particularly focused on language models.', 'features': ['Code generation and debugging assistance', 'Text summarization and paraphrasing', 'Dialogue generation and chatbot creation', 'Translation and text classification', 'Fine-tuning existing language models'], 'benefits': ['Easy-to-use interface for both beginners and experts', 'Modular design allows for custom